In [56]:
# import numba
# from numba import float64, int64
# from numba.typed import List
# from numba.experimental import jitclass
# from numba import njit
# # --- Node class ---
# node_spec = [
#     ('circle_id', int64),
#     ('SOC', float64),
#     ('x', float64),
#     ('y', float64)
# ]

# @jitclass(node_spec)
# class Node:
#     def __init__(self, circle_id, SOC, x, y):
#         self.circle_id = circle_id
#         self.SOC = SOC
#         self.x = x
#         self.y = y

#     def get_data(self):
#         return (self.circle_id, self.SOC, self.x, self.y)

# # --- Edge class ---
# edge_spec = [
#     ('length', float64),
#     ('risk', float64),
#     ('from_circle', int64),
#     ('from_SOC', float64),
#     ('to_circle', int64),
#     ('to_SOC', float64)
# ]

# @jitclass(edge_spec)
# class Edge:
#     def __init__(self, length, risk, from_circle, from_SOC, to_circle, to_SOC):
#         self.length = length
#         self.risk = risk
#         self.from_circle = from_circle
#         self.from_SOC = from_SOC
#         self.to_circle = to_circle
#         self.to_SOC = to_SOC

#     def get_nodes(self):
#         return ((self.from_circle, self.from_SOC),
#                 (self.to_circle, self.to_SOC))

# # Create example instances to extract their types.
# node_type = numba.typeof(Node(0, 0.0, 0.0, 0.0))
# edge_type = numba.typeof(Edge(0.0, 0.0, 0, 0.0, 0, 0.0))

# # --- Graph class ---
# graph_spec = [
#     ('nodes', numba.types.ListType(node_type)),
#     ('edges', numba.types.ListType(edge_type))
# ]

# @jitclass(graph_spec)
# class Graph:
#     def __init__(self):
#         # Initialize typed lists with explicit type information.
#         self.nodes = List.empty_list(node_type)
#         self.edges = List.empty_list(edge_type)

#     def add_node(self, node):
#         self.nodes.append(node)

#     def get_node(self, circle_id, SOC):
#         for i in range(len(self.nodes)):
#             n = self.nodes[i]
#             if n.circle_id == circle_id and n.SOC == SOC:
#                 return n
#         raise ValueError("Node with given circle_id and SOC not found")

#     def add_edge(self, edge):
#         self.edges.append(edge)

#     def get_edge(self, index):
#         return self.edges[index]

# @njit
# def build_graph(num_iter):
#     graph = Graph()  # Graph is already a jitclass

#     for i in range(num_iter):
#         node1 = Node(1, 75.0, 10.0, 20.0)
#         node2 = Node(2, 80.0, 15.0, 25.0)
#         graph.add_node(node1)
#         graph.add_node(node2)

#         # This linear search in get_node is still going to be slow for large lists
#         # but at least it will be compiled and run in nopython mode.
#         try:
#             n = graph.get_node(1, 75.0)
#         except ValueError as e:
#             # In nopython mode, raising exceptions is still costly.
#             pass

#         edge = Edge(10.0, 0.2, node1.circle_id, node1.SOC, node2.circle_id, node2.SOC)
#         graph.add_edge(edge)

#         e = graph.get_edge(0)

#     return graph

# def main():
#     # Call the jitted function
#     g = build_graph(100000)
#     # Possibly do more with g

# if __name__ == '__main__':
#     main()



In [57]:
# Import all required libraries
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import sympy
# from sympy import symbols, Eq, solve, simplify
import numpy as np
from utils import plot_map, graph_construction
import networkx as nx
import time

In [58]:
### Create the visibility graph

class graph_construction_discritization():
    
    def __init__(self, Map_qz, start, goal, q_min, q_max, alpha, beta, discritization_angle, discritization_charge):
        self.Map_qz = Map_qz
        self.start, self.goal = start, goal
        self.q_min, self.q_max = q_min, q_max
        self.alpha, self.beta = alpha, beta
        self.discritization_angle, self.discritization_charge = discritization_angle, discritization_charge
        self.SOC_discritization = np.arange(self.q_min, self.q_max + self.discritization_charge, self.discritization_charge)
        self.SOC_discritization_len = len(self.SOC_discritization)
    
    def creating_nodes(self):
        # Precompute angles in radians
        angles = np.deg2rad(np.arange(0, 360+self.discritization_angle, self.discritization_angle))
        angles_repeated = np.repeat(angles, self.SOC_discritization_len)

        # Precompute the number of points per circle
        num_points_per_circle = len(angles)
        num_points_per_circle_with_SOC = len(angles_repeated)

        # Preallocate the array
        node_position_discritization = np.zeros((len(self.Map_qz) * num_points_per_circle_with_SOC + 2*self.SOC_discritization_len, 4))
        SOC_discritization_repeated = np.tile(self.SOC_discritization, num_points_per_circle) 

        ### Adding the start and goal points
        node_position_discritization[:self.SOC_discritization_len, 0] = self.start[0]
        node_position_discritization[:self.SOC_discritization_len, 1] = self.start[1]
        node_position_discritization[:self.SOC_discritization_len, 2] = 0
        node_position_discritization[:self.SOC_discritization_len, 3] = self.SOC_discritization

        node_position_discritization[self.SOC_discritization_len:2*self.SOC_discritization_len, 0] = self.goal[0]
        node_position_discritization[self.SOC_discritization_len:2*self.SOC_discritization_len, 1] = self.goal[1]
        node_position_discritization[self.SOC_discritization_len:2*self.SOC_discritization_len, 2] = len(self.Map_qz)+1
        node_position_discritization[self.SOC_discritization_len:2*self.SOC_discritization_len, 3] = self.SOC_discritization

        ### Vectorized computation
        for i, (cx, cy, r) in enumerate(self.Map_qz):
            start_idx = i * num_points_per_circle_with_SOC + 2*self.SOC_discritization_len
            end_idx = start_idx + num_points_per_circle_with_SOC

            # Compute x and y positions using NumPy broadcasting
            x_vals = cx + r * np.cos(angles_repeated)
            y_vals = cy + r * np.sin(angles_repeated)

            # Assign values to the array
            node_position_discritization[start_idx:end_idx, 0] = x_vals
            node_position_discritization[start_idx:end_idx, 1] = y_vals
            node_position_discritization[start_idx:end_idx, 2] = i+1
            node_position_discritization[start_idx:end_idx, 3] = SOC_discritization_repeated
        
        self.node_position_discritization = node_position_discritization
        return node_position_discritization
    
    def point_to_segment_distance(self,px, py, x1, y1, x2, y2):
        """Compute the minimum distance from a point (px, py) to a finite line segment (x1, y1) - (x2, y2)."""
        A = px - x1
        B = py - y1
        C = x2 - x1
        D = y2 - y1

        dot = A * C + B * D
        len_sq = C * C + D * D
        param = -1 if len_sq == 0 else dot / len_sq  # Projection parameter

        if param < 0:  # Closest to (x1, y1)
            xx, yy = x1, y1
        elif param > 1:  # Closest to (x2, y2)
            xx, yy = x2, y2
        else:  # Closest to the actual segment
            xx, yy = x1 + param * C, y1 + param * D

        dx, dy = px - xx, py - yy
        return np.sqrt(dx * dx + dy * dy)

    def line_segment_intersects_circle(self, circle, point1, point2):
        """Check if the finite line segment (point1 to point2) intersects the circle."""
        center_x, center_y, radius = circle
        x1, y1 = point1
        x2, y2 = point2

        return self.point_to_segment_distance(center_x, center_y, x1, y1, x2, y2) < radius

    def external_edge_addition(self, node_index_i, node_index_j, visibility_graph):
        for i in range(node_index_i, node_index_i + int(self.SOC_discritization_len)):
            for j in range(node_index_j, node_index_j + int(self.SOC_discritization_len)):
                node_i = tuple(self.node_position_discritization[i])
                node_j = tuple(self.node_position_discritization[j])
                
                distance = np.sqrt((node_i[0] - node_j[0])**2 + (node_i[1] - node_j[1])**2)                    
                SOC_i, SOC_j = int( node_i[3]),  int(node_j[3])
                line_segment = {}
                
                feasiblilty = False
                ### Feasibility check
                if SOC_j > min(self.q_max, SOC_i + self.beta*distance):
                    feasiblilty = False
                
                elif (SOC_i - self.alpha*distance) < self.q_min and ( SOC_j <= SOC_i - self.alpha*distance):
                    feasiblilty = True
                    lamb = 1
                    line_segment = {"e": (0, lamb)}
                else:
                    lamb_1 = (self.q_max - SOC_i)/(self.beta*distance)
                    lamb_2 = (self.q_max - SOC_j)/(self.alpha*distance)
                    
                    if lamb_1 >= 0 and lamb_2 >= 0 and (lamb_1 + lamb_2 <= 1):
                        lamb = lamb_1 + self.alpha/(self.beta + self.alpha) * (1-lamb_1- lamb_2)
                        lamb_2_sw = lamb_2 + self.beta/(self.beta + self.alpha) * (1-lamb_1- lamb_2)
                        line_segment = {"g": (0, lamb_1) , "e": (lamb_1, lamb_2_sw) , "g": (lamb_2_sw, 1)}
                    else:
                        lamb = (SOC_j - SOC_i + self.alpha*distance)/((self.alpha + self.beta)*distance)
                        line_segment = {"g": (0, lamb) , "e": (lamb, 1)}
                    if lamb <= 1 and lamb >= 0:
                        feasiblilty = True
                
                if feasiblilty:
                    cost = lamb*distance
                    visibility_graph.add_edge(node_i, node_j, line_segment=line_segment, fuel_cost=cost, risk_cost=0, feasiblilty=feasiblilty, edge_type="external")
            
        return visibility_graph

    def internal_edge_addition(self, node_index_i, node_index_j, visibility_graph):
        node_index_i = node_index_i*self.SOC_discritization_len + self.SOC_discritization_len
        node_index_j = node_index_j*self.SOC_discritization_len + self.SOC_discritization_len
        
        for i in range(node_index_i, node_index_i + self.SOC_discritization_len):
            for j in range(node_index_j, node_index_j + self.SOC_discritization_len):
                node_i = tuple(self.node_position_discritization[i])
                node_j = tuple(self.node_position_discritization[j])
                # print(f"node i and node j {node_i}, {node_j}")

                distance = np.sqrt((node_i[0] - node_j[0])**2 + (node_i[1] - node_j[1])**2)
                # print(f"The circle index in: {node_i[2]}, {node_j[2]}")
                x_c, y_c, radius = self.Map_qz[int(node_i[2]-1)]
                pen_dist = sympy.sqrt(radius**2-(distance/2)**2)
                
                SOC_i, SOC_j = int( node_i[3]),  int(node_j[3])
                line_segment = {}
                
                feasiblilty = False
                ### Feasibility check
                if SOC_j > min(self.q_max, SOC_i + self.beta*distance):
                    feasiblilty = False
                
                elif (SOC_i - self.alpha*distance) < self.q_min and ( SOC_j <= SOC_i - self.alpha*distance):
                    feasiblilty = True
                    lamb = 1
                    line_segment = {"e": (0, lamb)}
                else:
                    lamb_1 = (self.q_max - SOC_i)/(self.beta*distance)
                    lamb_2 = (self.q_max - SOC_j)/(self.alpha*distance)
                    
                    if lamb_1 >= 0 and lamb_2 >= 0 and (lamb_1 + lamb_2 <= 1):
                        lamb = lamb_1 + self.alpha/(self.beta + self.alpha) * (1-lamb_1- lamb_2)
                        lamb_2_sw = lamb_2 + self.beta/(self.beta + self.alpha) * (1-lamb_1- lamb_2)
                        line_segment = {"g": (0, lamb_1) , "e": (lamb_1, lamb_2_sw) , "g": (lamb_2_sw, 1)}
                    else:
                        lamb = (SOC_j - SOC_i + self.alpha*distance)/((self.alpha + self.beta)*distance)
                        line_segment = {"g": (0, lamb) , "e": (lamb, 1)}
                    if lamb <= 1 and lamb >= 0:
                        feasiblilty = True
                
                if feasiblilty:
                    cost = lamb*distance
                    risk_cost = self.risk_cost_cal(line_segment, pen_dist, distance)
                    visibility_graph.add_edge(node_i, node_j, line_segment=line_segment, fuel_cost=cost, risk_cost=risk_cost, feasiblilty=feasiblilty, edge_type="internal")
            
        return visibility_graph

    def risk_cost_cal(self, line_segment, pen_dist, distance):
        risk_cost = 0
        for key, value in line_segment.items():
            if key == "e":
                risk_cost += 0
            elif key == "g":
                lamb_start, lamb_end = value[0], value[1]
                if lamb_start <= 0.5 and lamb_end >= 0.5:
                    risk_cost += abs(1/pen_dist*(sympy.atan((distance/2 - lamb_start*distance)/pen_dist) + sympy.atan((lamb_end*distance - distance/2)/pen_dist)))
                elif lamb_start >= 0.5 and lamb_end >= 0.5:
                    risk_cost += abs(1/pen_dist*(sympy.atan((lamb_end*distance-distance/2)/pen_dist) - sympy.atan((lamb_start*distance - distance/2)/pen_dist)))
                elif lamb_start <= 0.5 and lamb_end <= 0.5:
                    risk_cost += abs(1/pen_dist*(sympy.atan((distance/2- lamb_start*distance)/pen_dist) - sympy.atan((distance/2-lamb_end*distance)/pen_dist)))
        
        return risk_cost

In [59]:
# Define constants and initial conditions
Map_qz = [(0.0, 0.0, 6.0), (12.0, 10.0, 4.0)]  # QZ circles with (x, y, radius)
start = (-5, -5)  # Starting point
goal = (15, 15)   # Goal point

### The UAV characteristics
alpha, recharge_factor = 0.5, 2
beta = alpha/recharge_factor

q_min, q_max = 20, 100
discritization_angle = 5
discritization_charge = 10
SOC_discritization = np.arange(q_min, q_max+discritization_charge, discritization_charge)
SOC_discritization_len = len(SOC_discritization)
### Risk limit for each circle, temporary
Risk_limit = { circle: 30 for circle in Map_qz}

### Index Map
index_map = {i: circle for i, circle in enumerate(Map_qz)}

In [60]:
graph_object = graph_construction_discritization(Map_qz, start, goal, q_min, q_max, alpha, beta, discritization_angle, discritization_charge)
node_position_discritization = graph_object.creating_nodes()

visibility_graph = nx.DiGraph()

for i in range(0,node_position_discritization.shape[0], SOC_discritization_len):
    for j in range(i+SOC_discritization_len, node_position_discritization.shape[0], SOC_discritization_len):
        node_index_i = int(node_position_discritization[i][2])
        node_index_j = int(node_position_discritization[j][2])
        
        x_i, y_i = node_position_discritization[i][0], node_position_discritization[i][1]
        x_j, y_j = node_position_discritization[j][0], node_position_discritization[j][1]
        
        intersection_exists = False
        if (node_index_i == node_index_j) and node_index_i != 0 and node_index_i != len(Map_qz)+1:
            # print(f"The circle index out: {node_index_i}, {node_index_j}")
            visibility_graph = graph_object.internal_edge_addition(node_index_i, node_index_j, visibility_graph)
        else:
            for circle in Map_qz:
                intersection_exists = graph_object.line_segment_intersects_circle(circle, (x_i, y_i), (x_j, y_j))
                if intersection_exists:
                    break
            
            if not intersection_exists:
                visibility_graph = graph_object.external_edge_addition(node_index_i, node_index_j, visibility_graph)

C:\Users\Saurabh Belgaonkar\AppData\Local\Temp\ipykernel_27048\1329058432.py:105: RuntimeWarning: divide by zero encountered in scalar divide
  lamb_1 = (self.q_max - SOC_i)/(self.beta*distance)
C:\Users\Saurabh Belgaonkar\AppData\Local\Temp\ipykernel_27048\1329058432.py:106: RuntimeWarning: divide by zero encountered in scalar divide
  lamb_2 = (self.q_max - SOC_j)/(self.alpha*distance)
C:\Users\Saurabh Belgaonkar\AppData\Local\Temp\ipykernel_27048\1329058432.py:113: RuntimeWarning: invalid value encountered in scalar divide
  lamb = (SOC_j - SOC_i + self.alpha*distance)/((self.alpha + self.beta)*distance)
C:\Users\Saurabh Belgaonkar\AppData\Local\Temp\ipykernel_27048\1329058432.py:113: RuntimeWarning: divide by zero encountered in scalar divide
  lamb = (SOC_j - SOC_i + self.alpha*distance)/((self.alpha + self.beta)*distance)
C:\Users\Saurabh Belgaonkar\AppData\Local\Temp\ipykernel_27048\1329058432.py:105: RuntimeWarning: invalid value encountered in scalar divide
  lamb_1 = (self.q_